In [5]:
import numpy as np
import pandas as pd
import geopandas
import neilpy

geopandas.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [2]:
# Load data

# nodes must have a unique ID in the first column, and then longitudes and latitudes in columns 2 and 3.
# edges must have start and destination unique IDs that match those in nodes

nodes = pd.read_csv('data/nodes.csv',index_col=0)
edges = pd.read_csv('data/edges.csv')

E, meta = neilpy.imread('data/cusco_elevation.tif')
S = neilpy.slope(E,meta['cellsize'])

In [8]:
#%% DIRECT

gdf = neilpy.direct_routes(nodes,edges)
gdf.to_file('out/direct.shp')

In [9]:
#%% AREAL SLOPE EXAMPLE

label = 'tobler'
C = neilpy.cost_tobler_hiking_function(S)
gdf = neilpy.get_areal_routes(nodes,edges,C,meta,label=label)
gdf.to_file('out/' + label + '.shp')
gdf.to_file('out/' + label + '.kml',driver='KML')

label = 'rademaker'
C = neilpy.cost_rademaker(S)
gdf = neilpy.get_areal_routes(nodes,edges,C,meta,label=label)
gdf.to_file('out/' + label + '.shp')
gdf.to_file('out/' + label + '.kml',driver='KML')

Creating surface network for tobler
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.
Creating surface network for rademaker
Done creating surface network.
Calculating costs and routes.
Done calculating costs and routes.
Calculating costs and routes.
Done calculating costs and routes.


In [10]:
#%% LINEAR SLOPE EXAMPLE

label = 'pingel_linear_9.25'
scale_factor = 9.25

df = neilpy.create_raster_network(E)
df['slope'] = np.abs(df['source_value'] - df['target_value']) / (meta['cellsize']*df['distance'])  #dx/dz
df['slope'] = np.rad2deg(np.arctan(df['slope']))
df['slope'] = neilpy.cost_pingel_exponential(df['slope'],scale_factor)
df['weight'] = df['distance'] * df['slope']

gdf = neilpy.get_linear_routes(nodes, edges, df, meta, label=label)
gdf.to_file('out/' + label + '.shp')
gdf.to_file('out/' + label + '.kml',driver='KML')